In [1]:
!pip install -q recsim

In [ ]:
import gym
from recsim import document
from recsim.choice_model import MultinomialLogitChoiceModel
from recsim.simulator import environment
from recsim.simulator import recsim_gym

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from geopy.geocoders import Nominatim
from math import radians, sin, cos, sqrt, atan2

In [ ]:
tourist_spots_data = pd.read_csv("data.csv")
tourist_spots_data

,Tourist Spot,City,Location Lat,Location Long,Budget (INR),Opening Time,Closing Time,Seasonality
0,Adalaj Stepwell,Ahmedabad,23.1686,72.5901,0,06:00:00,18:00:00,All
1,Albert Hall Museum,Jaipur,26.9115,75.8194,100,09:00:00,17:00:00,All
2,Amber Fort,Jaipur,26.9855,75.8513,500,08:00:00,17:30:00,All
3,Anjuna Beach,Goa,15.5865,73.7398,0,00:00:00,23:59:59,All
4,Art of Living International Center,Bangalore,12.8614,77.7260,0,00:00:00,23:59:59,All
...,...,...,...,...,...,...,...,...
84,Taraporewala Aquarium,Mumbai,18.9569,72.8130,100,10:00:00,20:00:00,All
85,Tulip Garden,Kashmir,34.0689,74.7958,0,00:00:00,23:59:59,All
86,Victoria Memorial,Kolkata,22.5448,88.3426,20,09:00:00,17:00:00,All
87,Vidhana Soudha,Bangalore,12.9794,77.5901,0,09:00:00,17:00:00,All


In [ ]:
#tourist_spots_data['Opening Time'] = pd.to_datetime(tourist_spots_data['Opening Time']).dt.time
#tourist_spots_data['Closing Time'] = pd.to_datetime(tourist_spots_data['Closing Time']).dt.time
#tourist_spots_data

In [ ]:
tourist_spots_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Tourist Spot   89 non-null     object 
 1   City           89 non-null     object 
 2   Location Lat   89 non-null     float64
 3   Location Long  89 non-null     float64
 4   Budget (INR)   89 non-null     int64  
 5   Opening Time   89 non-null     object 
 6   Closing Time   89 non-null     object 
 7   Seasonality    89 non-null     object 
dtypes: float64(2), int64(1), object(5)
memory usage: 5.7+ KB


In [ ]:
tourist_spots_data['Budget (INR)'] = tourist_spots_data['Budget (INR)'].astype(int)
tourist_spots_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Tourist Spot   89 non-null     object 
 1   City           89 non-null     object 
 2   Location Lat   89 non-null     float64
 3   Location Long  89 non-null     float64
 4   Budget (INR)   89 non-null     int64  
 5   Opening Time   89 non-null     object 
 6   Closing Time   89 non-null     object 
 7   Seasonality    89 non-null     object 
dtypes: float64(2), int64(1), object(5)
memory usage: 5.7+ KB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
feedback_data = pd.read_csv('feedback.csv')
feedback_data

,user_id,tourist_spot,feedback,rating,date_of_feedback
0,1,Adalaj Stepwell,Great place to visit,5,2024-03-13
1,2,Albert Hall Museum,Interesting exhibits,4,2024-03-13
2,3,Amber Fort,Loved the architecture,5,2024-03-13
3,4,Anjuna Beach,Beautiful sunset view,4,2024-03-13
4,5,Art of Living International Center,Peaceful atmosphere,4,2024-03-13
5,6,Baga Beach,Fun activities,3,2024-03-13
6,7,Bangalore Palace,Historical gem,5,2024-03-13
7,8,Bannerghatta National Park,Amazing wildlife,4,2024-03-13
8,9,Basilica of Bom Jesus,Historical significance,5,2024-03-13
9,10,Belur Math,Spiritual experience,4,2024-03-13


In [ ]:
feedback_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   user_id           44 non-null     int64 
 1   tourist_spot      44 non-null     object
 2   feedback          44 non-null     object
 3   rating            44 non-null     int64 
 4   date_of_feedback  44 non-null     object
dtypes: int64(2), object(3)
memory usage: 1.8+ KB


In [ ]:
tourist_spots_data['City'].unique()

array(['Ahmedabad', 'Jaipur', 'Goa', 'Bangalore', 'Kolkata', 'Kashmir',
       'Hyderabad', 'Delhi', 'Mumbai', 'Chennai'], dtype=object)

In [ ]:
def get_location_coordinates(location_name):
    loc = Nominatim(user_agent="GetLoc")
    get_loc = loc.geocode(location_name)
    return get_loc.latitude, get_loc.longitude

In [ ]:
def calculate_distance(lat1, lon1, lat2, lon2):
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)

    R = 6371.0

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance


In [ ]:
def store_feedback(user_id, place, feedback, rating, date):
    feedback_data = pd.DataFrame({
        'user_id': [user_id],
        'place': [place],
        'feedback': [feedback],
        'rating': [rating],
        'date_of_feedback': [date]
    })
    feedback_data.to_csv("feedback.csv", mode='a', header=False, index=False)

In [ ]:
datetime.now().time()

datetime.time(9, 37, 40, 860412)

In [ ]:
class TravelEnv(gym.Env):
    def __init__(self, tourist_spots_data):
        super(TravelEnv, self).__init__()
        self.tourist_spots_data = tourist_spots_data
        self.current_state = None
        self.done = False

    def step(self, action):
        recommended_spot = self.tourist_spots_data.iloc[action]
        spot_name = recommended_spot['Tourist Spot']
        spot_opening_time = recommended_spot['Opening Time']
        spot_closing_time = recommended_spot['Closing Time']

        current_time = datetime.now().time()
        opening_time = datetime.strptime(spot_opening_time, "%H:%M:%S").time()
        closing_time = datetime.strptime(spot_closing_time, "%H:%M:%S").time()

        if opening_time <= current_time <= closing_time:
            self.current_state = spot_name
            reward = 1
        else:
            reward = -0.1  # Some penalty value

        # Check if episode is done (e.g., user reaches the budget limit)
        if self.user_budget <= 0:
            self.done = True

        return self.current_state, reward, self.done, {}

    def reset(self):
        self.current_state = None
        self.done = False
        return self.current_state


In [ ]:
def get_recommendation(user_id, place, budget, tourist_spots_data, user_feedback):

    # Check if the user has any previous feedback
    user_history = user_feedback[user_feedback['user_id'] == user_id]
    user_has_history = not user_history.empty

    # Filter tourist spots based on location
    place_filtered = tourist_spots_data[tourist_spots_data['City'] == place]

    # Filter spots based on budget
    affordable_spots = place_filtered[place_filtered['Budget (INR)'].astype(int) <= budget]

    # Filter spots based on opening hours
    current_time = datetime.now().time()
    open_spots = affordable_spots[affordable_spots.apply(lambda x:
                                                        datetime.strptime(x['Opening Time'], "%H:%M:%S").time() <= current_time <= datetime.strptime(x['Closing Time'], "%H:%M:%S").time(), axis=1)]


    if user_has_history:
        # Merge user's history with feedback from other users
        all_feedback = pd.merge(user_feedback, user_history, how='outer')
        feedback_scores = all_feedback.groupby('tourist_spot')['rating'].mean()
        recommended_spots = open_spots.merge(feedback_scores, left_on='Tourist Spot', right_index=True, how='left')
        recommended_spots = recommended_spots.sort_values(by='rating', ascending=False)
    else:
        # If user is new, recommend based on overall ratings
        recommended_spots = open_spots.sort_values(by='rating', ascending=False)


    if not recommended_spots.empty:
        top_recommendation = recommended_spots.iloc[0]['Tourist Spot']
        return top_recommendation
    else:
        return "No recommendations available within your budget and location."


In [ ]:
def main():
    env = TravelEnv(tourist_spots_data)
    choice_features = {
        "Tourist Spot": 0,
        "City": 1,
        "Location Lat": 2,
        "Location Long": 3,
        "Budget (INR)": 4,
        "Opening Time": 5,
        "Closing Time": 6
    }
    choice_model = MultinomialLogitChoiceModel(choice_features)
    doc = document.AbstractDocument
    recsim_env = recsim_gym.RecSimGymEnv(env, choice_model, doc)

    # Get user input
    user_id = int(input("Enter user ID: "))
    place = input("Enter desired place: ")
    budget = float(input("Enter budget (INR): "))

    # Get recommendation
    recommendation = get_recommendation(user_id, place, budget, tourist_spots_data, feedback_data)

    # Display recommendation to the user
    print("Your Recommendation:", recommendation)

    # Ask for feedback
    feedback = input("Provide feedback (Yes/No): ")
    if feedback.lower() == "yes":
        rating = int(input("Rate the recommendation (1-5): "))
        comment = input("Comments on this place: ")
        date_of_feedback = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        store_feedback(user_id, recommendation, comment, rating, date_of_feedback)

if __name__ == "__main__":
    main()


Enter user ID: 1
Enter desired place: Ahmedabad
Enter budget (INR): 100


/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


Your Recommendation: Adalaj Stepwell
Provide feedback (Yes/No): Yes
Rate the recommendation (1-5): 4
Comments on this place: It's a lovely place


In [ ]:
#if (place_filtered['Location Lat'] - lat) ** 2 + (place_filtered['Location Long'] - lon) ** 2 <= distance_threshold ** 2:
     #   filtered_spots = tourist_spots_data[(place_filtered['Location Lat'] - lat) ** 2 + (place_filtered['Location Long'] - lon) ** 2 <= distance_threshold ** 2]
      #  print(filtered_spots)